In [4]:
import sys
sys.path.insert(0, r'P:\pet ML')
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from data_preprocessing import preprocessing

In [2]:
prep = preprocessing()

In [6]:
prep.process()

Before norm - mean: 88.28278133333333, std: 62.39534902771433
After norm - mean: 1.5006662579253317e-17, std: 1.0
